In [1]:
import pandas as pd
import numpy as np
import csv
import statsmodels.api as sm
import os

In [2]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [3]:
import dotenv

# Load the environment variables
# (loads CENSUS_API_KEY from .env)
dotenv.load_dotenv()

False

In [4]:
%%R
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In addition: Warning messages:
1: package ‘readr’ was built under R version 4.2.3 
2: package ‘dplyr’ was built under R version 4.2.3 
3: package ‘stringr’ was built under R version 4.2.3 


In [5]:
# Open the file and turn it into a pandas dataframe
df = pd.read_csv('Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')

# Show the first 5 rows of the dataframe
print(df.head())

   RegionID  SizeRank  RegionName RegionType StateName State      City  \
0     91982         1       77494        zip        TX    TX      Katy   
1     61148         2        8701        zip        NJ    NJ  Lakewood   
2     91940         3       77449        zip        TX    TX      Katy   
3     62080         4       11368        zip        NY    NY  New York   
4     91733         5       77084        zip        TX    TX   Houston   

                                   Metro        CountyName     2000-01-31  \
0   Houston-The Woodlands-Sugar Land, TX  Fort Bend County  210077.000246   
1  New York-Newark-Jersey City, NY-NJ-PA      Ocean County  137227.265500   
2   Houston-The Woodlands-Sugar Land, TX     Harris County  103358.701090   
3  New York-Newark-Jersey City, NY-NJ-PA     Queens County  146558.052897   
4   Houston-The Woodlands-Sugar Land, TX     Harris County  102016.821579   

   ...     2023-05-31     2023-06-30     2023-07-31     2023-08-31  \
0  ...  474580.751007 

In [6]:
# Print the column names
print(df.columns)

Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'City', 'Metro', 'CountyName', '2000-01-31',
       ...
       '2023-05-31', '2023-06-30', '2023-07-31', '2023-08-31', '2023-09-30',
       '2023-10-31', '2023-11-30', '2023-12-31', '2024-01-31', '2024-02-29'],
      dtype='object', length=299)


In [7]:
# Keep only data with State as NJ
df = df[df['State'] == 'WA']

# Keep only columns including RegionName, StateName, City, Metro, CountyName, 31/12/2018, 31/12/2019, 31/12/2022, 31/12/2023
df = df[['RegionName', 'StateName', 'City', 'Metro', 'CountyName', '2018-12-31', '2019-12-31', '2022-12-31', '2023-12-31']]

In [8]:
# Ditch the rows with NaN values
df = df.dropna()

# Show how many rows are left
print(df.shape)

(399, 9)


In [9]:
df['RegionName'] = df['RegionName'].astype(str).str.zfill(5)

In [10]:
# Show the first 5 rows of RegionName
print(df['RegionName'].head())

70     99301
139    98052
148    98012
313    98682
477    98208
Name: RegionName, dtype: object


In [11]:
# Store all the RegionNames in a list
wa_zips = df['RegionName'].tolist()

In [12]:
%%R 

require('tidycensus')

# because it an environment variable, we don't have to 
# explicitly pass this string to R, it is readable here
# in this R cell.
census_api_key(Sys.getenv("97779d587a8fdba4e4f7f7d3eec838313d5b07ab"))

Loading required package: tidycensus
To install your API key for use in future sessions, run this function with `install = TRUE`.
In addition: Warning message:
package ‘tidycensus’ was built under R version 4.2.3 


In [13]:
%%R 
# Get variable from ACS
wa_census_data_2022 <- get_acs(geography = "zip code tabulation area",
                      variables = c(
                        total_population_2022 = "B01003_001",
                        median_room_2022 = "B25018_001",
                        median_built_2022 = "B25035_001",
                        median_income_2022 = "B06011_001",
                        bachelor_2022 = "B26106_005",
                        asian_2022 = "B03002_006", 
                        black_2022 = "B03002_004",
                        hispanic_2022 = "B03002_012"
                      ), 
                      year = 2022,
                      survey="acs5",
                      geometry=F)
wa_census_data_2022

# A tibble: 270,192 × 5
   GEOID NAME        variable              estimate   moe
   <chr> <chr>       <chr>                    <dbl> <dbl>
 1 00601 ZCTA5 00601 total_population_2022    16834   506
 2 00602 ZCTA5 00602 total_population_2022    37642   205
 3 00603 ZCTA5 00603 total_population_2022    49075   963
 4 00606 ZCTA5 00606 total_population_2022     5590   264
 5 00610 ZCTA5 00610 total_population_2022    25542   344
 6 00611 ZCTA5 00611 total_population_2022     1315   382
 7 00612 ZCTA5 00612 total_population_2022    63312  1805
 8 00616 ZCTA5 00616 total_population_2022     9625  1319
 9 00617 ZCTA5 00617 total_population_2022    22573   241
10 00622 ZCTA5 00622 total_population_2022     7577   979
# ℹ 270,182 more rows
# ℹ Use `print(n = ...)` to see more rows


Getting data from the 2018-2022 5-year ACS
In addition: Warning message:
• You have not set a Census API key. Users without a key are limited to 500
queries per day and may experience performance limitations.
ℹ For best results, get a Census API key at
http://api.census.gov/data/key_signup.html and then supply the key to the
`census_api_key()` function to use it throughout your tidycensus session.
This warning is displayed once per session. 


In [14]:
%R -i wa_zips

wa_zips

['99301',
 '98052',
 '98012',
 '98682',
 '98208',
 '99208',
 '98391',
 '98387',
 '98105',
 '98115',
 '98023',
 '98103',
 '98270',
 '98632',
 '98258',
 '99336',
 '98003',
 '98661',
 '98042',
 '98034',
 '98902',
 '98133',
 '98223',
 '98501',
 '98226',
 '98225',
 '98118',
 '98801',
 '98125',
 '98204',
 '98374',
 '99205',
 '98837',
 '99362',
 '99206',
 '98290',
 '98036',
 '98031',
 '98058',
 '98122',
 '98032',
 '98503',
 '98087',
 '98277',
 '98059',
 '98033',
 '98026',
 '98366',
 '98004',
 '98006',
 '98908',
 '98001',
 '98117',
 '98662',
 '98584',
 '98607',
 '98604',
 '98030',
 '98404',
 '98056',
 '98684',
 '98370',
 '98513',
 '98155',
 '99163',
 '98198',
 '98203',
 '98168',
 '98375',
 '98499',
 '99337',
 '98002',
 '99352',
 '98926',
 '98021',
 '99223',
 '98201',
 '98229',
 '98038',
 '98109',
 '98512',
 '99207',
 '98312',
 '98802',
 '98040',
 '98445',
 '98029',
 '98498',
 '98901',
 '98683',
 '98466',
 '98367',
 '98027',
 '98074',
 '98338',
 '98037',
 '98531',
 '98146',
 '98273',
 '98144',


In [15]:
%%R
# Remove the "ZCTA5 " prefix from the NAME column
wa_census_data_2022$NAME <- sub("ZCTA5 ", "", wa_census_data_2022$NAME)

# Show the first 5 rows of the wa_census_data_2022 dataframe
head(wa_census_data_2022)

# A tibble: 6 × 5
  GEOID NAME  variable              estimate   moe
  <chr> <chr> <chr>                    <dbl> <dbl>
1 00601 00601 total_population_2022    16834   506
2 00602 00602 total_population_2022    37642   205
3 00603 00603 total_population_2022    49075   963
4 00606 00606 total_population_2022     5590   264
5 00610 00610 total_population_2022    25542   344
6 00611 00611 total_population_2022     1315   382


In [16]:
%%R
# Keep only the rows with RegionName in wa_zips
wa_census_data_2022 <- wa_census_data_2022[wa_census_data_2022$NAME %in% wa_zips,]

# Show the first 5 rows of the wa_census_data_2022 dataframe
head(wa_census_data_2022)

# A tibble: 6 × 5
  GEOID NAME  variable              estimate   moe
  <chr> <chr> <chr>                    <dbl> <dbl>
1 98001 98001 total_population_2022    35842  1907
2 98002 98002 total_population_2022    36812  1637
3 98003 98003 total_population_2022    51329  1996
4 98004 98004 total_population_2022    39161  1830
5 98005 98005 total_population_2022    22030  1507
6 98006 98006 total_population_2022    39753  1924


In [17]:
%%R
# Ditch NAME and moe columns
wa_census_data_2022 <- wa_census_data_2022 %>%
  select(-NAME, -moe)

In [18]:
%%R
# Convert the long format to wide format
wa_census_data_2022_wide <- wa_census_data_2022 %>%
  pivot_wider(
    names_from = variable,  # Column that contains variable names
    values_from = estimate  # Column that contains the values
  )

# Show the first 5 rows of the nj_census_data_2022 dataframe
head(wa_census_data_2022_wide)

# A tibble: 6 × 9
  GEOID total_population_2022 median_room_2022 median_built_2022
  <chr>                 <dbl>            <dbl>             <dbl>
1 98001                 35842              6.2              1985
2 98002                 36812              4.5              1977
3 98003                 51329              4.5              1984
4 98004                 39161              3.8              1998
5 98005                 22030              4.8              1980
6 98006                 39753              7.6              1979
# ℹ 5 more variables: median_income_2022 <dbl>, bachelor_2022 <dbl>,
#   asian_2022 <dbl>, black_2022 <dbl>, hispanic_2022 <dbl>


In [19]:
%%R
df <- read.csv("Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")

# Fill the RegionName to 5 digits
df$RegionName <- as.character(df$RegionName)
df$RegionName <- sprintf("%05s", df$RegionName)

# Keep only data with State as NJ
df <- df[df$State == 'WA',]

# Show the first 5 rows of the dataframe
head(df)

    RegionID SizeRank RegionName RegionType StateName State      City
71    100179       72      99301        zip        WA    WA     Pasco
140    99534      145      98052        zip        WA    WA   Redmond
149    99500      154      98012        zip        WA    WA   Bothell
314    99961      320      98682        zip        WA    WA Vancouver
478    99631      487      98208        zip        WA    WA   Everett
608   100157      617      99208        zip        WA    WA   Spokane
                                  Metro       CountyName X2000.01.31
71               Kennewick-Richland, WA  Franklin County    138643.6
140         Seattle-Tacoma-Bellevue, WA      King County    293480.1
149         Seattle-Tacoma-Bellevue, WA Snohomish County    264402.9
314 Portland-Vancouver-Hillsboro, OR-WA     Clark County    155308.4
478         Seattle-Tacoma-Bellevue, WA Snohomish County    217152.8
608          Spokane-Spokane Valley, WA   Spokane County    139052.6
    X2000.02.29 X2000.03.31

In [20]:
%%R
# How many rows are left
nrow(df)

[1] 486


In [21]:
%%R

# Append the X2022.12.31 column to the ma_census_data_2022_wide dataframe
# With RegionName in df matching GEOID in ma_census_data_2022_wide. Only keep the rows with a match.

wa_census_data_2022_wide <- wa_census_data_2022_wide %>%
    inner_join(df, by = c('GEOID' = 'RegionName'))

# Show the first 5 rows of the nj_census_data_2022_wide dataframe
head(wa_census_data_2022_wide)

# A tibble: 6 × 307
  GEOID total_population_2022 median_room_2022 median_built_2022
  <chr>                 <dbl>            <dbl>             <dbl>
1 98001                 35842              6.2              1985
2 98002                 36812              4.5              1977
3 98003                 51329              4.5              1984
4 98004                 39161              3.8              1998
5 98005                 22030              4.8              1980
6 98006                 39753              7.6              1979
# ℹ 303 more variables: median_income_2022 <dbl>, bachelor_2022 <dbl>,
#   asian_2022 <dbl>, black_2022 <dbl>, hispanic_2022 <dbl>, RegionID <int>,
#   SizeRank <int>, RegionType <chr>, StateName <chr>, State <chr>, City <chr>,
#   Metro <chr>, CountyName <chr>, X2000.01.31 <dbl>, X2000.02.29 <dbl>,
#   X2000.03.31 <dbl>, X2000.04.30 <dbl>, X2000.05.31 <dbl>, X2000.06.30 <dbl>,
#   X2000.07.31 <dbl>, X2000.08.31 <dbl>, X2000.09.30 <dbl>, X2000.10.31 <dbl>,

In [22]:
%%R

# Create a new column named bachelor_rate_2022, which is the ratio of bachelor_2022 to total_population_2022
wa_census_data_2022_wide <- wa_census_data_2022_wide %>%
    mutate(bachelor_rate_2022 = bachelor_2022 / total_population_2022)

# Create a new column named asian_rate_2022, which is the ratio of asian_2022 to total_population_2022
wa_census_data_2022_wide <- wa_census_data_2022_wide %>%
    mutate(asian_rate_2022 = asian_2022 / total_population_2022)

# Create a new column named black_rate_2022, which is the ratio of black_2022 to total_population_2022
wa_census_data_2022_wide <- wa_census_data_2022_wide %>%
    mutate(black_rate_2022 = black_2022 / total_population_2022)

# Create a new column named hispanic_rate_2022, which is the ratio of hispanic_2022 to total_population_2022
wa_census_data_2022_wide <- wa_census_data_2022_wide %>%
    mutate(hispanic_rate_2022 = hispanic_2022 / total_population_2022)

In [23]:
%%R

# Create a new df, which includes the following columns from nj_census_data_2022_wide:
# GEOID, total_population_2022, median_room_2022, median_built_2022, median_income_2022, bachelor_rate_2022, asian_rate_2022, black_rate_2022, hispanic_rate_2022, X2022.12.31
df_final <- wa_census_data_2022_wide %>%
    select(GEOID, total_population_2022, median_room_2022, median_built_2022, median_income_2022, bachelor_rate_2022, asian_rate_2022, black_rate_2022, hispanic_rate_2022, X2022.12.31)

# Show the first 5 rows of the df_final dataframe
head(df_final)

# A tibble: 6 × 10
  GEOID total_population_2022 median_room_2022 median_built_2022
  <chr>                 <dbl>            <dbl>             <dbl>
1 98001                 35842              6.2              1985
2 98002                 36812              4.5              1977
3 98003                 51329              4.5              1984
4 98004                 39161              3.8              1998
5 98005                 22030              4.8              1980
6 98006                 39753              7.6              1979
# ℹ 6 more variables: median_income_2022 <dbl>, bachelor_rate_2022 <dbl>,
#   asian_rate_2022 <dbl>, black_rate_2022 <dbl>, hispanic_rate_2022 <dbl>,
#   X2022.12.31 <dbl>


In [24]:
%%R

# Run a multiple linear regression model with X2022.12.31 as the dependent variable
# and the other columns as the independent variables

model <- lm(X2022.12.31 ~ total_population_2022 + median_room_2022 + median_built_2022 + median_income_2022 + asian_rate_2022 + black_rate_2022 + hispanic_rate_2022, data = df_final)

# Show the summary of the model
summary(model)


Call:
lm(formula = X2022.12.31 ~ total_population_2022 + median_room_2022 + 
    median_built_2022 + median_income_2022 + asian_rate_2022 + 
    black_rate_2022 + hispanic_rate_2022, data = df_final)

Residuals:
    Min      1Q  Median      3Q     Max 
-796208  -86189  -12079   64640 2323291 

Coefficients:
                        Estimate Std. Error t value Pr(>|t|)    
(Intercept)            8.627e+05  1.398e+06   0.617   0.5375    
total_population_2022 -8.203e-01  6.836e-01  -1.200   0.2309    
median_room_2022       6.954e+04  1.122e+04   6.198 1.48e-09 ***
median_built_2022     -6.726e+02  7.117e+02  -0.945   0.3452    
median_income_2022     1.345e+01  9.641e-01  13.946  < 2e-16 ***
asian_rate_2022        1.295e+06  1.855e+05   6.978 1.32e-11 ***
black_rate_2022       -7.186e+05  3.099e+05  -2.319   0.0209 *  
hispanic_rate_2022     4.011e+04  6.557e+04   0.612   0.5411    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 190200 on 38